In [0]:

import importlib
import mlflow
import mlflow.pyfunc
import pandas as pd
from mlflow.tracking import MlflowClient
import azure_connector
import numpy as np

importlib.reload(azure_connector)
from azure_connector import AzureConnector

In [0]:
    scope = 'WellnessSync'
    server = dbutils.secrets.get(scope=scope, key='AZURE_SQL_SERVER')
    database = dbutils.secrets.get(scope=scope, key='AZURE_SQL_DATABASE')
    username = dbutils.secrets.get(scope=scope, key='AZURE_SQL_USERNAME')
    password = dbutils.secrets.get(scope=scope, key='AZURE_SQL_PASSWORD')

    connector = AzureConnector(server, database, username, password)


In [0]:
store_id = 8
df = connector.read_dataframe()
df = df.toPandas()
# df = connector.filter_dataframe(df,store_id)



In [0]:
print (df.display(5)) 

In [0]:
def make_future_dataframe(df, periods=30, freq='D'):
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)
        
    if 'ds' not in df.columns:
        raise ValueError("DataFrame must contain 'ds' column")
        
    df['ds'] = pd.to_datetime(df['ds'])
    last_date = df['ds'].max()
    
    future_dates = pd.date_range(
        start=last_date + pd.Timedelta(days=1),
        periods=periods,
        freq=freq
    )

    future_df = pd.DataFrame({
        'ds': future_dates,
        'y': np.nan  # NaN for unknown future values
    })
    
    return future_df

In [0]:
df = pd.read_csv ('/Workspace/Users/ryan@delve.systems/Prophet_AI/grouped_data.csv')


In [0]:
model_name = "WellnessSalesForecast_Base"
model_stage = "2" 
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_stage}")

In [0]:
#df.drop_duplicates(subset=['ds'], inplace=True)
future_dataframe = make_future_dataframe(df=df,periods=30)
forecast = model.predict(future_dataframe)

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,6))
plt.plot(forecast['ds'], forecast['yhat1'])
plt.xlabel('ds')
plt.ylabel('y')
plt.title('Forecast')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
import matplotlib.pyplot as plt

df_actual = pd.read_csv('/Workspace/Users/ryan@delve.systems/Prophet_AI/temp-June_2025.csv')
plt.figure(figsize=(20,6))
plt.plot(df_actual['ds'], df_actual['y'])
plt.xlabel('ds')
plt.ylabel('y')
plt.title('Actual')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
df.head()

In [0]:
forecast.display()

In [0]:
%restart_python

In [0]:
connector.write_dataframe(forecast,'forecast')